In [17]:
#imports
import math
import numpy as np
import random
import itertools
from random import choices
import cmath

In [18]:
#constants
GAIN_MATRIX = (
    (3.95370716e-13, 1.40831353e-12, 1.30345221e-13, 1.58616128e-12),
    (1.10454943e-11, 1.08797904e-13, 1.70626457e-11, 1.87910752e-13),
    (5.35349571e-12, 1.75915771e-12, 1.10933550e-11, 2.15002508e-12),
    (9.57092193e-12, 1.14512900e-11, 7.62246884e-13, 1.58625393e-11)
)

TARGET_BITRATE=13142071.76687619

In [19]:
def generate_random_solutions(powers, no_of_users, num_random_solutions):
    return np.random.choice(powers, size=(num_random_solutions, no_of_users))


In [20]:
powers = [0,2,4]
no_of_users = 4
num_random_solutions = 8
pop_size = generate_random_solutions(powers, no_of_users, num_random_solutions)
print(pop_size)

[[4 0 4 4]
 [0 4 0 2]
 [0 0 0 2]
 [0 0 0 2]
 [2 0 0 0]
 [2 0 2 4]
 [2 0 4 4]
 [4 2 2 4]]
